### Trainning model for flower image classification
### Model used VGG16 implement from keras library

In [1]:
print("Test Jupiter in Visual studio code")

Test Jupiter in Visual studio code


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator


In [3]:
data_dir = r"flowers/"
batch_size = 32
img_height = 224 
img_width = 224
img_d = 3

#### Pre-processing

In [4]:
train_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 15740 files belonging to 16 classes.
Using 12592 files for training.


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 15740 files belonging to 16 classes.
Using 3148 files for validation.


In [6]:
class_name = train_data.class_names
print(class_name)

['astilbe', 'bellflower', 'black_eyed_susan', 'calendula', 'california_poppy', 'carnation', 'common_daisy', 'coreopsis', 'daffodil', 'dandelion', 'iris', 'magnolia', 'rose', 'sunflower', 'tulip', 'water_lily']


In [7]:
# plt.figure(figsize=(10,10))
# for images, labels in train_data.take(1):
#     for i in range(16):
#         ax = plt.subplot(4,4,i+1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_name[labels[i]])
#         plt.axis("off")

In [8]:
### Get infor from first image
for image_batch, labels_batch in train_data:
    print(image_batch.shape)
    print(labels_batch.shape)
    break


(32, 224, 224, 3)
(32,)


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_data.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
normalization_layer = layers.Rescaling(1./255)

In [11]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [12]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [13]:
num_classes = len(class_name)
print(num_classes)

16


In [14]:
model = Sequential(
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(64, 3 ,padding="same", activation="relu"),
    layers.Conv2D(64, 3,padding="same", activation="relu"),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, padding="same", activation="relu"),
    layers.Conv2D(128, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),

    layers.Conv2D(256, 3, padding="same", activation="relu"),
    layers.Conv2D(256, 3, padding="same", activation="relu"),
    layers.Conv2D(256, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),

    layers.Conv2D(512, 3, padding="same", activation="relu"),
    layers.Conv2D(512, 3, padding="same", activation="relu"),
    layers.Conv2D(512, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),

    layers.Conv2D(512, 3, padding="same", activation="relu"),
    layers.Conv2D(512, 3, padding="same", activation="relu"),
    layers.Conv2D(512, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    
    layers.Dropout(0.4),

    layers.Flatten(),
    layers.Dense(units=1024,activation="relu"),
    layers.Dense(units=1024,activation="relu"),
    layers.Dense(units=num_classes, name="output"),
)



In [16]:
from keras.optimizers import adam_v2
opt = adam_v2(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

TypeError: 'module' object is not callable

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      4160      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     8320      
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     16512     
                                                      

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')


In [ ]:
hist = model.fit_generator(
                        steps_per_epoch=100,
                        generator=train_data, 
                        validation_data= val_ds, 
                        validation_steps=10,
                        epochs=100,
                        callbacks=[checkpoint,early]
                    )

Epoch 1/15
12592/12592 [==============================] - ETA: 0s - loss: 2.7782 - accuracy: 0.0644

: 

: 